In [1]:
import os
import tensorflow as tf
from keras import layers
from keras import Model
from os import getcwd
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD

C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\joshi\Anaconda3\envs\tfcpu\lib\site-packages\tensorflow\python\f

In [2]:
model = keras.applications.VGG16(input_shape = (224, 224, 3), 
                                include_top = True, 
                                weights = 'imagenet')


for layer in model.layers:
    layer.trainable = True
    
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0    

In [3]:
last_layer = model.get_layer('fc2')
last_output = last_layer.output

x = keras.layers.Dense(2,activation='softmax')(last_output)           

model = Model( model.input, x) 

model.compile(optimizer = Adam(lr=0.00001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [4]:
data_dir = "C:\\Users\\joshi\\OneDrive\\Desktop\\NIT Surat\\Modified COVID\\Data\\" # Your Dataset Location Path
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Val')
train_malaria_dir = os.path.join(train_dir, 'COVID') #Replace the folder names
train_uninfected_dir = os.path.join(train_dir, 'NON-COVID')
validation_malaria_dir = os.path.join(val_dir, 'COVID')
validation_uninfected_dir = os.path.join(val_dir, 'NON-COVID')

train_malaria_fnames = os.listdir(train_malaria_dir)
train_uninfected_fnames = os.listdir(train_uninfected_dir)
validation_malaria_fnames = os.listdir(validation_malaria_dir)
validation_uninfected_fnames = os.listdir(validation_uninfected_dir)

print(len(train_malaria_fnames))
print(len(validation_malaria_fnames))
print(len(train_uninfected_fnames))
print(len(validation_uninfected_fnames))

955
60
1170
58


In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   
                                   )


train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=8,
                                                    class_mode='categorical',
                                                    target_size=(224, 224)
                                                    )
validation_datagen = ImageDataGenerator(rescale=1./255)
 

validation_generator = validation_datagen.flow_from_directory(val_dir,
                                                              batch_size=8,
                                                              class_mode='categorical',
                                                              target_size=(224, 224))

Found 2125 images belonging to 2 classes.
Found 118 images belonging to 2 classes.


In [6]:
history = model.fit_generator(train_generator,
            validation_data = validation_generator,
            epochs = 20)


Epoch 1/20
266/266 [==============================] - 1749s 7s/step - loss: 0.5839 - acc: 0.7002 - val_loss: 0.4248 - val_acc: 0.7034
Epoch 2/20
266/266 [==============================] - 1751s 7s/step - loss: 0.2490 - acc: 0.8936 - val_loss: 0.3775 - val_acc: 0.7966
Epoch 3/20
266/266 [==============================] - 1785s 7s/step - loss: 0.0783 - acc: 0.9689 - val_loss: 0.2814 - val_acc: 0.7458
Epoch 4/20
266/266 [==============================] - 1652s 6s/step - loss: 0.0242 - acc: 0.9925 - val_loss: 3.0259 - val_acc: 0.7458
Epoch 5/20
266/266 [==============================] - 1650s 6s/step - loss: 0.0546 - acc: 0.9779 - val_loss: 0.7666 - val_acc: 0.7458
Epoch 6/20
266/266 [==============================] - 1638s 6s/step - loss: 0.0041 - acc: 1.0000 - val_loss: 1.7965 - val_acc: 0.7797
Epoch 7/20
266/266 [==============================] - 1640s 6s/step - loss: 0.0408 - acc: 0.9868 - val_loss: 1.5002 - val_acc: 0.7712
Epoch 8/20
266/266 [==============================] - 1644s 6

In [7]:
test_dir = os.path.join(data_dir, 'Test')
test_malaria_dir = os.path.join(test_dir, 'COVID') #Replace the folder names
test_uninfected_dir = os.path.join(test_dir, 'NON-COVID')
test_malaria_fnames = os.listdir(test_malaria_dir)
test_uninfected_fnames = os.listdir(test_uninfected_dir)
print(len(test_malaria_fnames))
print(len(test_uninfected_fnames))

98
105


In [8]:
testGenerator = ImageDataGenerator().flow_from_directory(test_dir, 
                                                      batch_size=8,
                                                    class_mode='categorical',shuffle=False,target_size=(224,224)
                                                      )

Found 203 images belonging to 2 classes.


In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = 8
num_of_test_samples = 203
predictions = model.predict_generator(testGenerator)

y_pred = np.argmax(predictions, axis=1)

true_classes = testGenerator.classes

class_labels = list(testGenerator.class_indices.keys())   

print(class_labels)

print(confusion_matrix(testGenerator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

['COVID', 'NON-COVID']
[[75 23]
 [11 94]]
              precision    recall  f1-score   support

       COVID       0.87      0.77      0.82        98
   NON-COVID       0.80      0.90      0.85       105

    accuracy                           0.83       203
   macro avg       0.84      0.83      0.83       203
weighted avg       0.84      0.83      0.83       203



In [10]:
from sklearn import metrics
metrics.roc_auc_score(y_pred, testGenerator.classes)

0.8377559133373087

In [11]:
metrics.accuracy_score(testGenerator.classes, y_pred)

0.8325123152709359